In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [3]:
model = ChatOpenAI(temperature=0, model="gpt-5-nano")

In [ ]:
# 원하는 출력 구조 정의 pydantic
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [5]:
question = "지구 온난화의 심각성에 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"title": "Description", "description": "주제에 대한 간결한 설명", "type": "string"}, "hashtags": {"title": "Hashtags", "description": "해시태그 형식의 키워드(2개 이상)", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

chain.invoke({"question": question})

{'description': '지구 온난화는 온실가스 증가로 지구 평균 기온이 상승해 극한 기상 현상 증가, 해수면 상승, 빙하 감소, 생태계 교란 등 사회경제와 생태계에 심각한 영향을 주며 대응이 시급합니다.',
 'hashtags': '#지구온난화 #기후변화 #해수면상승'}

## Pydantic 없이 JSON 형태로 응답받기

In [10]:
model = ChatOpenAI(temperature=0, model="gpt-5-nano")

# 질문
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON Parser
parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요"),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

# 지시사항 주입
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인
chain = prompt | model | parser

response = chain.invoke({"question": question})

print(response)

{'description': '지구 온난화는 인간 활동으로 배출된 온실가스가 대기 중에 열을 가두어 지구 평균 기온이 장기간 상승하는 현상입니다. 주요 원인은 화석연료 연소, 산림 파괴, 산업 공정으로 인한 이산화탄소(CO2), 메탄(CH4), 아산화질소(N2O) 등의 배출이며, 대기 중 가스 농도가 높아질수록 온실효과가 강화됩니다. 그 결과로 폭염과 열파의 빈도가 늘고 빙하가 녹아 해수면이 상승하며, 강수 패턴과 생태계가 변합니다. 따라서 배출 감축, 재생에너지 확대, 에너지 효율 개선, 탄소 포집·저장, 산림 보전 등의 조치가 필요합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#재생에너지', '#탄소중립', '#기후정책']}
